In [3]:
import pandas as pd
import difflib

In [4]:
dfDicoBuch = pd.read_csv("../data/dictionary/1757_Buchanan-J.csv" , delimiter=";", encoding="utf8")
pronChar = pd.read_csv("../data/dictionary/Buchanan_PronChar_counts.csv", delimiter=";", encoding="utf8").dropna()

In [5]:
#Transformation qui permet de rechercher la correspondance des lettres de Buchanan
def traduction(text):
    newWord = ""
    for eachLetter in text:
        if pronChar["char"].isin([eachLetter]).any():
            newWord += pronChar.loc[pronChar["char"] == eachLetter]["graph"].item()
    return newWord

Création de la colonne motVedette grâce à la fonction de traduction établie précédemment

In [6]:
dfDicoBuch["motVedette"] = dfDicoBuch.apply(lambda x: traduction(x['pron']),1)

In [7]:
dicoBigi = dfDicoBigi = pd.read_csv("../data/dictionay/2015_Bigi-B.dict", delimiter=" \[\] ", encoding="utf8", engine='python', names=['word','sampa'], skiprows=8 )
dfMappingSampaIpa = pd.read_csv("../data/dictionay/sampa-ipa.txt", delimiter="  *", encoding="utf8", engine='python', names=['sampa','ipa'])
dfMappingSampaIpa.drop_duplicates(subset=['sampa'], keep='first', inplace=True) 

Fonction de traduction du Sampa à l'IPA qui sera appelée pour tout le dictionnaire Bigi

In [8]:
def sampa2ipa(strSampa, dfMapping):
    ipaRes = ""
    
    listValSampa = strSampa.split(" ")
    #print(listValSampa)
    for valSampa in listValSampa:
        dfRowFinded = dfMapping[dfMapping['sampa']==valSampa]

        if len(dfRowFinded==1):
            ipaRes+=dfRowFinded.iloc[0]['ipa']
        else:
            ipaRes+='?'
    return ipaRes

On ajoute la colonne IPA au dictionnaire Bigi

In [10]:
dfMerged = dfDicoBuch.merge(dicoBigi, left_on="motVedette", right_on="word", how="left")

In [12]:
dfMerged

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,word,sampa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,abacus,{ b @ k @ s
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft,NaN,NaN
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance,NaN,NaN
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,abandon,@ b { n d @ n
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,abandoned,@ b { n d @ n d
...,...,...,...,...,...,...,...,...,...,...,...
15795,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography,NaN,NaN
15796,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes,NaN,NaN
15797,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric,NaN,NaN
15798,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist,NaN,NaN


In [13]:
dfMerged["sampa"] = dfMerged["sampa"].astype(str)
dfMerged['ipa'] = dfMerged.apply(lambda x: sampa2ipa(x['sampa'], dfMappingSampaIpa),1)

In [17]:
dfMerged.loc[pd.isna(dfMerged["word"])]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,word,sampa,ipa
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft,NaN,nan,?
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance,NaN,nan,?
5,2,ăbāsʹe,F.,NaN,NaN,v.,7.0,19,abase,NaN,nan,?
6,3,ăbāʹſ_e_ment,F.,NaN,NaN,n.,8.0,19,abasement,NaN,nan,?
10,2,ăʹbbĕſs,NaN,NaN,NaN,n.,12.0,19,abbess,NaN,nan,?
...,...,...,...,...,...,...,...,...,...,...,...,...
15795,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography,NaN,nan,?
15796,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes,NaN,nan,?
15797,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric,NaN,nan,?
15798,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist,NaN,nan,?


Un peu plus de 5000 mots n'ont pas été trouvés dans le dictionnaire Bigi. Il y a par exemple abaft ou abintestate qui semblent être des mots mal traduits dans le dictionnaire de Buchanan. On propose ainsi d'utiliser une bibliothèque qui permet de trouver l'occurence la plus proche. Exemple ci-dessous avec le mot abase.

In [18]:
listWords = list(map(str, list(dicoBigi["word"])))
difflib.get_close_matches("abase", listWords, n=1)

['ambase']

On créé une nouvelle colonne : closest_word

In [19]:
#La fonction get_close_matches retourne l'occurence la plus proche dans le bag of words du dictionnaire bigi
#Si le mot est trouvé, il est par conséquent l'occurence la plus proche
def closest_word(text):
    return difflib.get_close_matches(text, listWords, n=1)

In [27]:
dfDicoBuch['closestWord'] = dfDicoBuch.apply(lambda x: closest_word(x['motVedette']),1)

In [43]:
dfDicoBuch = dfDicoBuch[dfDicoBuchv2['closestWord'].apply(len)>=1]

In [47]:
dfDicoBuch["motVedette"] = dfDicoBuch.apply(lambda x: x["closestWord"][0],1)
dfDicoBuch.pop("closestWord")

<ipython-input-47-60e5a5a65f46>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDicoBuch["motVedette"] = dfDicoBuch.apply(lambda x: x["closestWord"][0],1)


0             [abacus]
1              [sabat]
2           [ambiance]
3            [abandon]
4          [abandoned]
             ...      
15795     [topography]
15796    [sporophytes]
15797     [sophomoric]
15798      [zoologist]
15799         [optica]
Name: closestWord, Length: 15797, dtype: object

In [48]:
dfDicoBuch

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,sabat
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,ambiance
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned
...,...,...,...,...,...,...,...,...,...
15795,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,topography
15796,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,sporophytes
15797,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,sophomoric
15798,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zoologist


On peut ainsi ré-effectuer le merge sur cette colonne

In [49]:
dfMerged = dfDicoBuch.merge(dicoBigi, left_on="motVedette", right_on="word", how="left")

In [50]:
dfMerged

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,word,sampa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,abacus,{ b @ k @ s
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,sabat,sabat,s A b A t
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,ambiance,ambiance,{ m b i: @ n s
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,abandon,@ b { n d @ n
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,abandoned,@ b { n d @ n d
...,...,...,...,...,...,...,...,...,...,...,...
15792,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,topography,topography,t @ p A g r\ @ f i:
15793,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,sporophytes,sporophytes,s p O: r\ f aI t s
15794,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,sophomoric,sophomoric,s A f @U m A r\ i k
15795,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zoologist,zoologist,z @U A l @ dZ @ s t


In [51]:
dfMerged["sampa"] = dfMerged["sampa"].astype(str)
dfMerged['ipa'] = dfMerged.apply(lambda x: sampa2ipa(x['sampa'], dfMappingSampaIpa),1)

In [52]:
dfMerged

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,word,sampa,ipa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,abacus,{ b @ k @ s,æbəkəs
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,sabat,sabat,s A b A t,sɑbɑt
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,ambiance,ambiance,{ m b i: @ n s,æmbiːəns
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,abandon,@ b { n d @ n,əbændən
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,abandoned,@ b { n d @ n d,əbændənd
...,...,...,...,...,...,...,...,...,...,...,...,...
15792,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,topography,topography,t @ p A g r\ @ f i:,təpɑgɹəfiː
15793,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,sporophytes,sporophytes,s p O: r\ f aI t s,spOːɹf?ts
15794,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,sophomoric,sophomoric,s A f @U m A r\ i k,sɑfoʊmɑɹik
15795,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zoologist,zoologist,z @U A l @ dZ @ s t,zoʊɑlə?əst


Nous avons réussi à conserver 15797 sur 15800. 3 enregistrements n'ont pas pu être traduits ce qui a permis de conserver de nombreux enregistrements dans notre dictionnaire. On enregistre ce dataframe en csv.

In [53]:
dfMerged.to_csv("../data/dataframe/BuchananRich.csv", sep=";",encoding="utf8",index=False)